In [1]:
# Settings for autoreloading.

%load_ext autoreload
%autoreload 2

In [19]:
import onnxruntime as ort
import numpy       as np
import scipy       as sp

import torch
import glob
import os

from torchinfo import summary
from PIL       import Image
from torch     import cuda

from src.model.model import FireSegmentationModel
from typing          import Tuple

In [20]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
BASE_DIR = '.'

In [22]:
ORIGINAL_SHAPE = (3840, 2160)
RESIZE_SHAPE   = (512, 512)
BATCH_SIZE     = 1

In [30]:
from torch.nn.functional import softmax



class FireSegmentationModelImproved(FireSegmentationModel):
    def __init__(self, input_size: Tuple[int, int], device: str, softmax_flag = True):
        super().__init__(input_size, device)
        self.softmax_flag = softmax_flag
    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        y = super().forward(x)
        return softmax(y, dim=1) if self.softmax_flag  else y

In [33]:
SOFTMAX_FLAG = True

model = FireSegmentationModelImproved(RESIZE_SHAPE, DEVICE, SOFTMAX_FLAG)

In [34]:
summary(model.eval(), input_size=(BATCH_SIZE, 3, *RESIZE_SHAPE))

Layer (type:depth-idx)                                       Output Shape              Param #
FireSegmentationModelImproved                                [1, 2, 512, 512]          --
├─Encoder: 1-1                                               [1, 16, 256, 256]         --
│    └─DCNN: 2-1                                             [1, 16, 256, 256]         --
│    │    └─Sequential: 3-1                                  [1, 16, 256, 256]         464
│    │    └─Sequential: 3-2                                  [1, 16, 256, 256]         464
│    │    └─Sequential: 3-3                                  [1, 24, 128, 128]         7,880
│    │    └─Sequential: 3-4                                  [1, 40, 64, 64]           52,312
│    │    └─Sequential: 3-5                                  [1, 960, 16, 16]          2,910,832
│    └─ASPP: 2-2                                             [1, 256, 16, 16]          --
│    │    └─Sequential: 3-6                                  [1, 256, 16, 16]  

In [35]:
from src.training.utils import Checkpoint

# Set the checkpoint monitor.
checkpoint = Checkpoint(os.path.join(BASE_DIR, 'model', 'checkpoints.pth'))

In [36]:
checkpoint.load_best_weights(model)
model = model.eval();

### Экспортируем модель в формат ONNX

In [43]:
onnx_dir        = "onnx"
onnx_name       = f"current_model{'_softmax' if SOFTMAX_FLAG else ''}.onnx"
onnx_model_path = os.path.join(onnx_dir, onnx_name)

input_tensor = torch.randn(1, 3, 512, 512).cuda()

if not os.path.exists(onnx_dir):
    os.makedirs(onnx_dir)

torch.onnx.export(
    model,                              # model to export
    (input_tensor,),         # inputs of the model,
    onnx_model_path,         # filename of the ONNX model
    input_names=["input"],   # Rename inputs for the ONNX model
    output_names=["output"]  # Rename outputs for the ONNX model
)

print(os.path.join(onnx_dir, file_name))

SyntaxError: unmatched ')' (2709163347.py, line 3)

### Проверяем работу базовой модели

In [ ]:
# Define paths
model_path  = "current_model_softmax.onnx"
images_path = "data_fire/Images/*.jpg"  # Adjust the glob pattern as needed
    
# Load model
model = load_model(model_path)

# Get input shape from the model
input_shape = model.get_inputs()[0].shape  # Usually (1, C, H, W)
    
# Load and preprocess images
image_paths = glob.glob(images_path)[:5]

preprocessed_images = [preprocess_image(image_path, input_shape[1:]) for image_path in image_paths]
    
# Perform inference
predictions = infer_images(model, preprocessed_images)
    
# Display results
for image_path, prediction in zip(image_paths, predictions):
    print(f"Image:      {image_path}")
    print(f"Prediction: {prediction}")